In [3675]:
import yfinance as yf
import pandas as pd
import math
import numpy_financial as npf
import statistics
import datetime
from dateutil.relativedelta import relativedelta
from pandas.core.frame import DataFrame
from scipy.stats import gmean


# 設定定期投入金額、起始結束日期、投資組合配比

In [3676]:
PMT=1000
start='2020-1-1'
end='2020-12-31'
ratio=[0.5,0.5]
stock=['KO','PYPL']


In [3677]:
stock_name=[]
stock_list=[]
start_dt=datetime.datetime.strptime(start,"%Y-%m-%d")
start_dt=(start_dt-relativedelta(months=+1)).strftime("%Y-%m-%d")
end_dt=datetime.datetime.strptime(end,"%Y-%m-%d")
end_dt=(end_dt+datetime.timedelta(days=1)).strftime("%Y-%m-%d")
for i in range(len(stock)):
    price=yf.download(stock[i],start=start_dt,end=end_dt)
    stock_list.append(price)
    
for i in stock_list:
    i.drop(["Open"],axis=1,inplace=True)
    i.drop(["High"],axis=1,inplace=True)
    i.drop(["Low"],axis=1,inplace=True)
    i.drop(["Close"],axis=1,inplace=True)
    i.drop(["Volume"],axis=1,inplace=True)

    
if len(stock)==1:
    price=stock_list[0].copy()
    price.rename(columns={'Adj Close':stock[0]},inplace=True)
else:
    price=stock_list[0].copy()
    price.rename(columns={'Adj Close':stock[0]},inplace=True)
    for i in range(1,len(stock)):
        price=pd.merge(price,stock_list[i],left_index=True,right_index=True,how='inner')
        price.rename(columns={'Adj Close':stock[i]},inplace=True)
price

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,KO,PYPL
Date,,
2019-12-02,48.562840,106.209999
2019-12-03,48.608021,105.589996
2019-12-04,49.050732,105.000000
2019-12-05,48.960384,104.589996
2019-12-06,49.168182,104.339996
...,...,...
2020-12-24,49.935024,238.639999
2020-12-28,50.607807,235.729996
2020-12-29,50.579773,231.029999


In [3678]:
stock_df=price.copy()
stock_m_df=stock_df.resample('BM').last()
stock_m_df=stock_m_df.loc[start_dt:end]
# dfm=round(dfm,2)
stock_m_df

,KO,PYPL
Date,,
2019-12-31,50.008427,108.169998
2020-01-31,52.764091,113.889999
2020-02-28,48.327930,107.989998
2020-03-31,40.330269,95.739998
2020-04-30,41.824993,123.000000
2020-05-29,42.545010,155.009995
2020-06-30,41.092125,174.229996
2020-07-31,43.446560,196.070007
2020-08-31,45.552666,204.139999


In [3679]:
TotalCost_list=[]
for i in range(1,len(stock_m_df)+1):
    TotalCost=PMT
    TotalCost_list.append(TotalCost)
stock_m_df['total_cost']=TotalCost_list

In [3680]:
# 各支股票股數計算
unit_name_list=[]
for k in range(len(stock_price_dfm.columns[0:len(stock_list)])):
    unit_list=[]
    unit_name_list.append("unit"+str(k+1))
    unit_list=[PMT*ratio[k]/stock_m_df[stock_m_df.columns[k]].iloc[0]]
    for i in stock_m_df[stock_m_df.columns[k]].iloc[1:]:
        unit=unit_list[-1]
        unit=unit
        unit_list.append(unit)
    stock_m_df[unit_name_list[k]]=unit_list

stock_m_df.head(1)

,KO,PYPL,total_cost,unit1,unit2
Date,,,,,
2019-12-31,50.008427,108.169998,1000,9.998315,4.622354


In [3681]:
# 各支股票淨值計算
preNAV_name_list=[]
for k in range(len(stock)):
    preNAV_list=[0]
    preNAV_name_list.append("preNAV"+str(k+1))
    for i,j in enumerate(stock_m_df[stock_m_df.columns[k]].iloc[1:]):
        preNAV=stock_m_df[unit_name_list[k]].iloc[i]*j
        preNAV_list.append(preNAV)
    stock_m_df[preNAV_name_list[k]]=preNAV_list

stock_m_df.head(1)

,KO,PYPL,total_cost,unit1,unit2,preNAV1,preNAV2
Date,,,,,,,
2019-12-31,50.008427,108.169998,1000,9.998315,4.622354,0.0,0.0


In [3682]:
stock_m_df["final"]=stock_m_df[preNAV_name_list[0]].copy()
stock_m_df["final"][0]=PMT
for i in range(1,len(stock)):
    stock_m_df["final"]+=stock_m_df[preNAV_name_list[i]]


# stock_m_df["final"]=stock_m_df[preNAV_name_list[0]].copy()
# stock_m_df["final"][0]=PMT
# for i in range(1,len(stock)):
#     stock_m_df["final"]+=stock_m_df[preNAV_name_list[i]]
# # for i in range(len(dfm)):
#     dfm["final"][i]+=PMT
stock_m_df.head(10)

C:\Users\PHIL\AppData\Local\Temp\ipykernel_17460\141262415.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_m_df["final"][0]=PMT


,KO,PYPL,total_cost,unit1,unit2,preNAV1,preNAV2,final
Date,,,,,,,,
2019-12-31,50.008427,108.169998,1000,9.998315,4.622354,0.000000,0.000000,1000.000000
2020-01-31,52.764091,113.889999,1000,9.998315,4.622354,527.552005,526.439869,1053.991874
2020-02-28,48.327930,107.989998,1000,9.998315,4.622354,483.197870,499.167975,982.365844
2020-03-31,40.330269,95.739998,1000,9.998315,4.622354,403.234730,442.544141,845.778871
2020-04-30,41.824993,123.000000,1000,9.998315,4.622354,418.179454,568.549515,986.728969
2020-05-29,42.545010,155.009995,1000,9.998315,4.622354,425.378406,716.511034,1141.889440
2020-06-30,41.092125,174.229996,1000,9.998315,4.622354,410.852007,805.352679,1216.204687
2020-07-31,43.446560,196.070007,1000,9.998315,4.622354,434.392389,906.304940,1340.697329
2020-08-31,45.552666,204.139999,1000,9.998315,4.622354,455.449899,943.607298,1399.057197


In [3683]:
# monthly_return
mreturn_df=stock_m_df.copy()
monthly_return=[(mreturn_df["final"][0]/PMT)-1]
for i in range(1,len(mreturn_df)):
    mreturn=((mreturn_df["final"][i])/mreturn_df["final"][i-1])-1
    monthly_return.append(mreturn*100)

stock_m_df["monthly return(%)"]=monthly_return

CAGR=round(((stock_m_df['final'][-1]/stock_m_df['total_cost'][0])**(1/((len(stock_m_df)-1)/12))-1)*100,2)

In [3684]:
def MDD(dr):
#     dr=df.pct_change(1)
    r=dr.add(1).cumprod()
    dd=r.div(r.cummax()).sub(1)
    mdd=dd.min()
    end=dd.idxmin()
    start=r.loc[:end[0]].idxmax()
    days=end-start
    return mdd[0], start[0], end[0], days[0]

# MAX Drawdown


In [3685]:
mdd=(stock_m_df["monthly return(%)"][0:]/100).copy()
mdd=mdd.to_frame()
MDD(mdd)


(-0.19754706654437415,
 Timestamp('2020-01-31 00:00:00'),
 Timestamp('2020-03-31 00:00:00'),
 Timedelta('60 days 00:00:00'))

# stdev

In [3691]:
stdev_df=stock_m_df.copy()
risk=stdev_df["monthly return(%)"][1:].std()
risk_Y=risk*12**0.5
risk_Y=round(risk_Y,2)
ar=stdev_df["monthly return(%)"][1:].mean()*12
std=risk_Y
# Sharpe
sp=ar/std
sp=round(sp,2)
# Sortino
sortino=[]
for i in range(1,len(stdev_df)):
    if stdev_df["monthly return(%)"][i]<0:
        sortino.append(stdev_df["monthly return(%)"][i])
    else:
        sortino.append(0)

sttd_D=DataFrame(sortino).std()*12**0.5
stt=ar/sttd_D
stt=round(stt,3)

print("年化標準差:",risk_Y,"%")
print('Sharpe ratio:',sp)
print('sortino ratio:',stt[0])


年化標準差: 32.82 %
Sharpe ratio: 1.6
sortino ratio: 3.533


In [3687]:
stock_m_df_c=stock_m_df.copy()
stock_m_df_c['final']=round(stock_m_df_c['final'])

stock_m_df_c

,KO,PYPL,total_cost,unit1,unit2,preNAV1,preNAV2,final,monthly return(%)
Date,,,,,,,,,
2019-12-31,50.008427,108.169998,1000,9.998315,4.622354,0.000000,0.000000,1000.0,0.000000
2020-01-31,52.764091,113.889999,1000,9.998315,4.622354,527.552005,526.439869,1054.0,5.399187
2020-02-28,48.327930,107.989998,1000,9.998315,4.622354,483.197870,499.167975,982.0,-6.795691
2020-03-31,40.330269,95.739998,1000,9.998315,4.622354,403.234730,442.544141,846.0,-13.903881
2020-04-30,41.824993,123.000000,1000,9.998315,4.622354,418.179454,568.549515,987.0,16.665124
2020-05-29,42.545010,155.009995,1000,9.998315,4.622354,425.378406,716.511034,1142.0,15.724730
2020-06-30,41.092125,174.229996,1000,9.998315,4.622354,410.852007,805.352679,1216.0,6.508095
2020-07-31,43.446560,196.070007,1000,9.998315,4.622354,434.392389,906.304940,1341.0,10.236159
2020-08-31,45.552666,204.139999,1000,9.998315,4.622354,455.449899,943.607298,1399.0,4.352949


In [3688]:

print("========================================================")
print("標的:",stock)
print("回測區間:",start,'~',end)
print("總損益:",stock_m_df_c['final'][-1])
print("CAGR:",CAGR,'%')
print("標準差:","{:.2f}".format(risk_Y),"%")
print('Sharpe ratio:',sp)
print('sortino ratio:',stt[0])
print('最大回撤(MAX Drawdown):',round(MDD(mdd)[0]*100,2),'%')
print('最大回撤區間:',MDD(mdd)[1].strftime('%Y-%m-%d'),'~',MDD(mdd)[2].strftime('%Y-%m-%d'))
print("========================================================")


標的: ['KO', 'PYPL']
回測區間: 2020-1-1 ~ 2020-12-31
總損益: 1595.0
CAGR: 59.49 %
標準差: 32.82 %
Sharpe ratio: 1.6
sortino ratio: 3.106
最大回撤(MAX Drawdown): -19.75 %
最大回撤區間: 2020-01-31 ~ 2020-03-31


In [3689]:
# return_name_list=[]
# cc=[]
# for k in range(len(dfm.columns[0:len(stock_list)])):
#     return_name_list.append("return"+str(k+1)+'(%)')
# #     dfm[cc[k]]=(dfm[aa[k]]/(dfm['total_cost']*ratio[k]-PMT*ratio[k])-1)*100
#     dfm[return_name_list[k]]=round((dfm[preNAV_name_list[k]]/(dfm['total_cost']*ratio[k]-PMT*ratio[k])-1)*100,2)

# dfm.head()

In [3690]:
# T_R=TR/100
# print('總回報',TR,'%')
# months=len(dfm)
# print("共有:",months,'月')
# IRR=(1+T_R)**(12/months)-1
# print('內部報酬率(IRR):',round(IRR*100,2),'%')